In [1]:
! rm -r *
! gdown 172vOcunNd08k83-0rwZVOYo45LSKYS8v
! sudo apt install rar
! rar x Genre\ Classification\ Dataset.rar
! rm Genre\ Classification\ Dataset.rar
# ! wget https://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove.6B.zip
# ! rm glove.6B.50d.txt
# ! rm glove.6B.100d.txt
# ! rm glove.6B.200d.txt
# ! rm glove.6B.zip

Downloading...
From: https://drive.google.com/uc?id=172vOcunNd08k83-0rwZVOYo45LSKYS8v
To: /content/Genre Classification Dataset.rar
100% 33.7M/33.7M [00:02<00:00, 16.4MB/s]
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 341 kB of archives.
After this operation, 1,202 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 rar amd64 2:5.5.0-1.1 [341 kB]
Fetched 341 kB in 2s (218 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to fr

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim.downloader as api
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
train_data = []
with open("./Genre Classification Dataset/train_data.txt") as f:
    for line in f.readlines():
        temp = line.strip().split(" ::: ")
        train_data.append({"text": temp[3].strip(), "movie_name": temp[1].strip(), "genre": temp[2].strip()})
train_df = pd.DataFrame(train_data)
train_df

,text,movie_name,genre
0,Listening in to a conversation between his doc...,Oscar et la dame rose (2009),drama
1,A brother and sister with a past incestuous re...,Cupid (1997),thriller
2,As the bus empties the students for their fiel...,"Young, Wild and Wonderful (1980)",adult
3,To help their unemployed father make ends meet...,The Secret Sin (1915),drama
4,The film's title refers not only to the un-rec...,The Unrecovered (2007),drama
...,...,...,...
54209,This short-lived NBC live sitcom centered on B...,"""Bonino"" (1953)",comedy
54210,The NEXT Generation of EXPLOITATION. The siste...,Dead Girls Don't Cry (????),horror
54211,"Ze bestaan echt, is a stand-up comedy about gr...",Ronald Goedemondt: Ze bestaan echt (2008),documentary
54212,Walter and Vivian live in the country and have...,Make Your Own Bed (1944),comedy


In [4]:
test_data = []
with open("./Genre Classification Dataset/test_data_solution.txt") as f:
    for line in f.readlines():
        temp = line.strip().split(" ::: ")
        test_data.append({"text": temp[3].strip(), "movie_name": temp[1].strip(), "genre": temp[2].strip()})
test_df = pd.DataFrame(test_data)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        54214 non-null  object
 1   movie_name  54214 non-null  object
 2   genre       54214 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [6]:
train_df['genre'].value_counts()

,count
genre,
drama,13613
documentary,13096
comedy,7447
short,5073
horror,2204
thriller,1591
action,1315
western,1032
reality-tv,884


In [7]:
train_df['text_char_length'] = train_df['text'].map(lambda x: len(x))
train_df.describe()

,text_char_length
count,54214.000000
mean,599.452429
std,446.026620
min,41.000000
25%,324.000000
50%,463.000000
75%,712.000000
max,10503.000000


In [8]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

X = "Movie Name: " + train_df['movie_name'] + '\nDescription:\n' + train_df['text']
X = X.apply(preprocess_text)
y = train_df['genre']

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

X_test = "Movie Name: " + test_df['movie_name'] + '\nDescription:\n' + test_df['text']
X_test = X_test.apply(preprocess_text)
y_test = test_df['genre']
y_test = label_encoder.transform(y_test)

In [ ]:
glove_model = api.load("glove-wiki-gigaword-300")
word2vec_model = api.load("word2vec-google-news-300")
fasttext_model = api.load("fasttext-wiki-news-subwords-300")
bert_model = SentenceTransformer('bert-base-uncased')
roberta_model = SentenceTransformer('roberta-base')
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

In [ ]:
def average_word_vectors(sentence, model, vector_size):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        raise Exception("Empty text")
        # return np.zeros(vector_size)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_train_glove = X_train.apply(lambda x: average_word_vectors(x, glove_model, 300))
X_train_word2vec = X_train.apply(lambda x: average_word_vectors(x, word2vec_model, 300))
X_train_fasttext = X_train.apply(lambda x: average_word_vectors(x, fasttext_model, 300))
X_train_bert = X_train.apply(lambda x: bert_model.encode(x))
X_train_roberta = X_train.apply(lambda x: roberta_model.encode(x))

X_val_tfidf = tfidf_vectorizer.transform(X_val).toarray()
X_val_glove = X_val.apply(lambda x: average_word_vectors(x, glove_model, 300))
X_val_word2vec = X_val.apply(lambda x: average_word_vectors(x, word2vec_model, 300))
X_val_fasttext = X_val.apply(lambda x: average_word_vectors(x, fasttext_model, 300))
X_val_bert = X_val.apply(lambda x: bert_model.encode(x))
X_val_roberta = X_val.apply(lambda x: roberta_model.encode(x))

X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()
X_test_glove = X_test.apply(lambda x: average_word_vectors(x, glove_model, 300))
X_test_word2vec = X_test.apply(lambda x: average_word_vectors(x, word2vec_model, 300))
X_test_fasttext = X_test.apply(lambda x: average_word_vectors(x, fasttext_model, 300))
X_test_bert = X_test.apply(lambda x: bert_model.encode(x))
X_test_roberta = X_test.apply(lambda x: roberta_model.encode(x))

In [27]:
X_train_glove = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_train_glove.to_list()])
X_val_glove = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_val_glove.to_list()])
X_test_glove = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_test_glove.to_list()])

X_train_word2vec = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_train_word2vec.to_list()])
X_val_word2vec = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_val_word2vec.to_list()])
X_test_word2vec = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_test_word2vec.to_list()])

X_train_fasttext = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_train_fasttext.to_list()])
X_val_fasttext = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_val_fasttext.to_list()])
X_test_fasttext = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_test_fasttext.to_list()])

X_train_bert = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_train_bert.to_list()])
X_val_bert = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_val_bert.to_list()])
X_test_bert = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_test_bert.to_list()])

X_train_roberta = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_train_roberta.to_list()])
X_val_roberta = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_val_roberta.to_list()])
X_test_roberta = np.concatenate([np.expand_dims(vector, axis=0) for vector in X_test_roberta.to_list()])

## TF-IDF

In [49]:
embedding_dim = X_train_tfidf.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_tfidf, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_tfidf, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_tfidf, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 2.7575, Train Accuracy: 50.8997, Val Loss: 2.3956, Val Accuracy: 54.6000
Epoch 2/20, Train Loss: 2.2259, Train Accuracy: 54.6873, Val Loss: 2.1252, Val Accuracy: 55.4235
Epoch 3/20, Train Loss: 2.0372, Train Accuracy: 55.7670, Val Loss: 2.0009, Val Accuracy: 56.5647
Epoch 4/20, Train Loss: 1.9280, Train Accuracy: 57.7168, Val Loss: 1.9169, Val Accuracy: 58.1412
Epoch 5/20, Train Loss: 1.8463, Train Accuracy: 59.7552, Val Loss: 1.8516, Val Accuracy: 60.0706
Epoch 6/20, Train Loss: 1.7795, Train Accuracy: 61.7168, Val Loss: 1.7980, Val Accuracy: 61.5412
Epoch 7/20, Train Loss: 1.7226, Train Accuracy: 63.3953, Val Loss: 1.7528, Val Accuracy: 62.7765
Epoch 8/20, Train Loss: 1.6732, Train Accuracy: 64.9056, Val Loss: 1.7144, Val Accuracy: 63.8588
Epoch 9/20, Train Loss: 1.6301, Train Accuracy: 66.1062, Val Loss: 1.6811, Val Accuracy: 64.5529
Epoch 10/20, Train Loss: 1.5916, Train Accuracy: 67.2537, Val Loss: 1.6524, Val Accuracy: 65.4588
Epoch 11/20, Train Loss: 1.55

## Glove

In [50]:
embedding_dim = X_train_glove.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_glove, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_glove, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_glove, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 2.2642, Train Accuracy: 48.5575, Val Loss: 1.9685, Val Accuracy: 56.2000
Epoch 2/20, Train Loss: 1.8597, Train Accuracy: 59.1593, Val Loss: 1.7790, Val Accuracy: 60.8824
Epoch 3/20, Train Loss: 1.7160, Train Accuracy: 63.3422, Val Loss: 1.6737, Val Accuracy: 64.2235
Epoch 4/20, Train Loss: 1.6289, Train Accuracy: 65.8879, Val Loss: 1.6049, Val Accuracy: 65.7294
Epoch 5/20, Train Loss: 1.5705, Train Accuracy: 67.5811, Val Loss: 1.5595, Val Accuracy: 66.8471
Epoch 6/20, Train Loss: 1.5288, Train Accuracy: 68.7021, Val Loss: 1.5250, Val Accuracy: 67.7294
Epoch 7/20, Train Loss: 1.4975, Train Accuracy: 69.5074, Val Loss: 1.5004, Val Accuracy: 68.7294
Epoch 8/20, Train Loss: 1.4734, Train Accuracy: 70.3304, Val Loss: 1.4804, Val Accuracy: 69.1529
Epoch 9/20, Train Loss: 1.4542, Train Accuracy: 70.8584, Val Loss: 1.4653, Val Accuracy: 69.5647
Epoch 10/20, Train Loss: 1.4384, Train Accuracy: 71.3127, Val Loss: 1.4521, Val Accuracy: 70.0118
Epoch 11/20, Train Loss: 1.42

## Word2vec

In [51]:
embedding_dim = X_train_word2vec.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_word2vec, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_word2vec, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_word2vec, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 2.4301, Train Accuracy: 46.1298, Val Loss: 2.1809, Val Accuracy: 52.6353
Epoch 2/20, Train Loss: 2.0971, Train Accuracy: 52.9794, Val Loss: 2.0260, Val Accuracy: 54.6000
Epoch 3/20, Train Loss: 1.9676, Train Accuracy: 55.7286, Val Loss: 1.9171, Val Accuracy: 56.6118
Epoch 4/20, Train Loss: 1.8725, Train Accuracy: 58.0383, Val Loss: 1.8348, Val Accuracy: 58.7882
Epoch 5/20, Train Loss: 1.7991, Train Accuracy: 60.3009, Val Loss: 1.7699, Val Accuracy: 61.2471
Epoch 6/20, Train Loss: 1.7404, Train Accuracy: 62.1003, Val Loss: 1.7189, Val Accuracy: 63.0588
Epoch 7/20, Train Loss: 1.6927, Train Accuracy: 63.7316, Val Loss: 1.6760, Val Accuracy: 64.2353
Epoch 8/20, Train Loss: 1.6533, Train Accuracy: 64.8643, Val Loss: 1.6409, Val Accuracy: 64.9294
Epoch 9/20, Train Loss: 1.6200, Train Accuracy: 65.7375, Val Loss: 1.6116, Val Accuracy: 66.0353
Epoch 10/20, Train Loss: 1.5919, Train Accuracy: 66.6077, Val Loss: 1.5860, Val Accuracy: 66.6118
Epoch 11/20, Train Loss: 1.56

## FastText

In [52]:
embedding_dim = X_train_fasttext.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_fasttext, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_fasttext, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_fasttext, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 2.6204, Train Accuracy: 39.4484, Val Loss: 2.3326, Val Accuracy: 44.3765
Epoch 2/20, Train Loss: 2.2751, Train Accuracy: 50.6254, Val Loss: 2.2357, Val Accuracy: 51.8588
Epoch 3/20, Train Loss: 2.2012, Train Accuracy: 52.3982, Val Loss: 2.1718, Val Accuracy: 52.9176
Epoch 4/20, Train Loss: 2.1426, Train Accuracy: 53.0177, Val Loss: 2.1171, Val Accuracy: 53.5529
Epoch 5/20, Train Loss: 2.0911, Train Accuracy: 53.6490, Val Loss: 2.0686, Val Accuracy: 54.1765
Epoch 6/20, Train Loss: 2.0449, Train Accuracy: 54.4071, Val Loss: 2.0251, Val Accuracy: 55.5176
Epoch 7/20, Train Loss: 2.0039, Train Accuracy: 55.2861, Val Loss: 1.9859, Val Accuracy: 55.9412
Epoch 8/20, Train Loss: 1.9665, Train Accuracy: 56.0590, Val Loss: 1.9508, Val Accuracy: 57.0941
Epoch 9/20, Train Loss: 1.9329, Train Accuracy: 56.9381, Val Loss: 1.9186, Val Accuracy: 57.5059
Epoch 10/20, Train Loss: 1.9021, Train Accuracy: 57.7729, Val Loss: 1.8900, Val Accuracy: 58.0824
Epoch 11/20, Train Loss: 1.87

## BERT

In [53]:
embedding_dim = X_train_bert.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_bert, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_bert, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_bert, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 1.8081, Train Accuracy: 61.0649, Val Loss: 1.5097, Val Accuracy: 69.5647
Epoch 2/20, Train Loss: 1.4292, Train Accuracy: 72.2183, Val Loss: 1.3807, Val Accuracy: 73.2588
Epoch 3/20, Train Loss: 1.3442, Train Accuracy: 74.5310, Val Loss: 1.3343, Val Accuracy: 74.8353
Epoch 4/20, Train Loss: 1.3052, Train Accuracy: 75.4307, Val Loss: 1.3103, Val Accuracy: 75.2118
Epoch 5/20, Train Loss: 1.2810, Train Accuracy: 76.2389, Val Loss: 1.2980, Val Accuracy: 75.7059
Epoch 6/20, Train Loss: 1.2647, Train Accuracy: 76.8584, Val Loss: 1.2894, Val Accuracy: 76.0471
Epoch 7/20, Train Loss: 1.2517, Train Accuracy: 77.0236, Val Loss: 1.2824, Val Accuracy: 76.2706
Epoch 8/20, Train Loss: 1.2417, Train Accuracy: 77.5015, Val Loss: 1.2772, Val Accuracy: 76.4588
Epoch 9/20, Train Loss: 1.2319, Train Accuracy: 77.6608, Val Loss: 1.2696, Val Accuracy: 76.9529
Epoch 10/20, Train Loss: 1.2251, Train Accuracy: 78.1416, Val Loss: 1.2701, Val Accuracy: 76.4824
Epoch 11/20, Train Loss: 1.21

## Roberta

In [54]:
embedding_dim = X_train_roberta.shape[1]
num_classes = len(label_encoder.classes_)
batch_size = 128
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_roberta, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_roberta, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_roberta, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = torch.nn.Linear(embedding_dim, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0

    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

    train_accuracy = correct / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_labels).sum().item()

    val_accuracy = correct / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

model.eval()
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()

test_accuracy = correct / len(test_loader)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/20, Train Loss: 2.0282, Train Accuracy: 54.4897, Val Loss: 1.7444, Val Accuracy: 63.3412
Epoch 2/20, Train Loss: 1.6271, Train Accuracy: 66.7670, Val Loss: 1.5433, Val Accuracy: 69.1882
Epoch 3/20, Train Loss: 1.4833, Train Accuracy: 70.6165, Val Loss: 1.4506, Val Accuracy: 71.2706
Epoch 4/20, Train Loss: 1.4096, Train Accuracy: 72.7286, Val Loss: 1.3972, Val Accuracy: 72.9294
Epoch 5/20, Train Loss: 1.3659, Train Accuracy: 73.8230, Val Loss: 1.3638, Val Accuracy: 74.1294
Epoch 6/20, Train Loss: 1.3359, Train Accuracy: 74.6313, Val Loss: 1.3442, Val Accuracy: 74.4000
Epoch 7/20, Train Loss: 1.3137, Train Accuracy: 75.1622, Val Loss: 1.3294, Val Accuracy: 74.6588
Epoch 8/20, Train Loss: 1.2969, Train Accuracy: 75.9558, Val Loss: 1.3244, Val Accuracy: 74.8118
Epoch 9/20, Train Loss: 1.2841, Train Accuracy: 76.1209, Val Loss: 1.3099, Val Accuracy: 75.4353
Epoch 10/20, Train Loss: 1.2726, Train Accuracy: 76.5546, Val Loss: 1.3036, Val Accuracy: 75.7176
Epoch 11/20, Train Loss: 1.26